In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
def select(sql):
    return pd.io.sql.read_sql(sql, con = engine) 

#схема структуры базы данных  
![Схема структуры базы данных](db.png)

### Задание 1:
- Посчитайте, сколько книг вышло после 1 января 2000 года;

Будем считать что по условию 1 января включено в промежуток.

In [10]:
sql = '''SELECT COUNT(*) FROM books t
WHERE t.publication_date >= DATE('2000-01-01')'''
select(sql)

,count
0,821


In [11]:
#если мы не уверены в отсутствии дубликатов , но уверены что у каждой книги уникальный ID можем сделать так:
sql = '''SELECT COUNT(DISTINCT(t.book_id)) FROM books t
WHERE t.publication_date >= DATE('2000-01-01')'''
select(sql)

,count
0,821


In [12]:
sql = '''SELECT COUNT(DISTINCT(t.book_id)) FROM books t '''
select(sql)

,count
0,1000


Из 1000 книг в таблице, 821 таблица выпущена в 21 веке.

### Задание 2:
- Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [13]:
sql = '''
SELECT t1.title, t1.book_id, t3.rating_avg, t4.review_numb
FROM books t1
LEFT JOIN
    (SELECT
    t.book_id,
    ROUND(AVG(t.rating),2) AS rating_avg
    FROM ratings t
    GROUP BY book_id) as t3
ON t1.book_id = t3.book_id
LEFT JOIN
    (SELECT 
    t.book_id,
    COUNT(t.review_id) AS review_numb
    FROM reviews t
    GROUP BY  book_id) as t4
ON t1.book_id = t4.book_id
'''

In [14]:
select(sql)

,title,book_id,rating_avg,review_numb
0,The Body in the Library (Miss Marple #3),652,4.50,2.0
1,Galápagos,273,4.50,2.0
2,A Tree Grows in Brooklyn,51,4.25,5.0
3,Undaunted Courage: The Pioneering First Missio...,951,4.00,2.0
4,The Prophet,839,4.29,4.0
...,...,...,...,...
995,Alice in Wonderland,64,4.23,4.0
996,A Woman of Substance (Emma Harte Saga #1),55,5.00,2.0
997,Christine,148,3.43,3.0
998,The Magicians' Guild (Black Magician Trilogy #1),790,3.50,2.0


### Задание 3:
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [15]:
sql = '''
SELECT publisher, books_numb
FROM
    (SELECT *
    FROM
        (SELECT
        t.publisher_id,
        COUNT(DISTINCT(t.book_id)) AS books_numb
        FROM books t
        WHERE t.num_pages > 50 
        GROUP BY t.publisher_id) AS f
        LEFT JOIN 
            (SELECT * FROM publishers) AS pub
    ON f.publisher_id= pub.publisher_id) AS tt
ORDER BY books_numb DESC
LIMIT 1
'''

In [16]:
select(sql)

,publisher,books_numb
0,Penguin Books,42


### Задание №4:

- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [19]:
sql = '''
SELECT t5.author, t5.avg FROM
    (SELECT * FROM
        (SELECT t3.author_id, AVG(t3.avg) FROM
            (SELECT * FROM 
                (SELECT t1.book_id, t1.author_id, AVG(t1.rating), COUNT(t1.rating_id)
                FROM
                    (SELECT t.book_id, author_id, rating_id, rating
                    FROM books t
                    LEFT JOIN ratings r
                    ON t.book_id = r.book_id) as t1 
                GROUP BY t1.book_id, t1.author_id) as t2
            WHERE t2.count >= 50) as t3
        GROUP BY t3.author_id) as t4
    LEFT JOIN authors a
    ON t4.author_id = a.author_id
    ORDER BY t4.avg DESC
    LIMIT 1) as t5
    
    
'''

select(sql)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


### Задание №5:
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [20]:
sql = '''
SELECT ROUND(AVG(count_rew),2) FROM
    (SELECT * FROM
        (SELECT  rt.username, COUNT(DISTINCT(rt.rating_id)) as count_rat
        FROM ratings rt
        GROUP BY rt.username) as t
    WHERE t.count_rat > 50) as t1
    LEFT JOIN 
        (SELECT rev.username, COUNT(rev.review_id) as count_rew
        FROM reviews rev
        GROUP BY rev.username) as t2 
ON t1.username = t2.username
'''

In [21]:
select(sql)

,round
0,24.33


Среднее количество обзоров от книгоманов, которые оставили более 50 оценок на прочитаные книги - 24,33.  
Таким образом, почти 50% поставленных оценок сопровождается рецензией.